## Create image datasets


### Image folder

`ImageFolder` automatically infers the class labels of your dataset based on the directory name. Store your dataset in a directory structure like:

```bash
folder/train/dog/golden_retriever.png
folder/train/dog/german_shepherd.png
folder/train/dog/chihuahua.png

folder/train/cat/maine_coon.png
folder/train/cat/bengal.png
folder/train/cat/birman.png
```

Then users can load your `dataset` by specifying `imagefolder` in `load_dataset()` and the directory in `data_dir`:

```python
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir="/path/to/folder")
```

You can also use `imagefolder` to load datasets involving multiple splits.

```bash
folder/train/dog/golden_retriever.png
folder/train/cat/maine_coon.png
folder/test/dog/german_shepherd.png
folder/test/cat/bengal.png
```

If there is additional information you’d like to include about your dataset, like text captions or bounding boxes, add it as a `metadata.csv` file in your folder

```bash
folder/train/metadata.csv
folder/train/0001.png
folder/train/0002.png
folder/train/0003.png
```

You can also zip your images:

```bash
folder/metadata.csv
folder/train.zip
folder/test.zip
folder/valid.zip
```

Your `metadata.csv` file must have a `file_name` column which links image files with their metadata:

```bash
file_name,additional_feature
0001.png,This is a first value of a text feature you added to your images
0002.png,This is a second value of a text feature you added to your images
0003.png,This is a third value of a text feature you added to your images
```

or using `metadata.jsonl`:

```json
{"file_name": "0001.png", "additional_feature": "This is a first value of a text feature you added to your images"}
{"file_name": "0002.png", "additional_feature": "This is a second value of a text feature you added to your images"}
{"file_name": "0003.png", "additional_feature": "This is a third value of a text feature you added to your images"}
```

### Image captioning

Image captioning datasets have text describing an image. An example `metadata.csv` may look like:

```bash
file_name,text
0001.png,This is a golden retriever playing with a ball
0002.png,A german shepherd
0003.png,One chihuahua
```
Load the dataset with `ImageFolder`, and it will create a text column for the image captions:

```bash
dataset = load_dataset("imagefolder", data_dir="/path/to/folder", split="train")
dataset[0]["text"]
```

### Object detection

Object detection datasets have bounding boxes and categories identifying objects in an image. An example `metadata.jsonl` may look like:

```jsonl
{"file_name": "0001.png", "objects": {"bbox": [[302.0, 109.0, 73.0, 52.0]], "categories": [0]}}
{"file_name": "0002.png", "objects": {"bbox": [[810.0, 100.0, 57.0, 28.0]], "categories": [1]}}
{"file_name": "0003.png", "objects": {"bbox": [[160.0, 31.0, 248.0, 616.0], [741.0, 68.0, 202.0, 401.0]], "categories": [2, 2]}}
```

Load the dataset with `ImageFolder`, and it will create a objects column with the bounding boxes and the categories:

```json
dataset = load_dataset("imagefolder", data_dir="/path/to/folder", split="train")
dataset[0]["objects"]
```

return 

```bash
{"bbox": [[302.0, 109.0, 73.0, 52.0]], "categories": [0]}
```

## Load Text data

By default, 🤗 Datasets samples a text file `line by line` to build the dataset.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("text", data_files={"train": ["my_text_1.txt", "my_text_2.txt"], "test": "my_test_file.txt"})

dataset = load_dataset("text", data_dir="path/to/text/dataset")

To sample a text file by `paragraph` or even an `entire document`, use the `sample_by parameter`:

In [ ]:
dataset = load_dataset("text", data_files={"train": "my_train_file.txt", "test": "my_test_file.txt"}, sample_by="paragraph")

dataset = load_dataset("text", data_files={"train": "my_train_file.txt", "test": "my_test_file.txt"}, sample_by="document")

You can also use `grep patterns` to load specific files:

In [ ]:
from datasets import load_dataset
c4_subset = load_dataset("allenai/c4", data_files="en/c4-train.0000*-of-01024.json.gz")

To load remote text files via HTTP, pass the URLs instead:

In [ ]:
dataset = load_dataset("text", data_files="https://huggingface.co/datasets/lhoestq/test/resolve/main/some_text.txt")

### Process text data

#### Map

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"]), batched=True)
dataset[0]

```json
{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 
 'label': 1, 
 'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102], 
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
```

In [ ]:
dataset = dataset.map(lambda examples: tokenizer(examples["text"], return_tensors="np"), batched=True)

#### Align

The `align_labels_with_mapping()` function aligns a dataset label id with the label name. 

Not all 🤗 Transformers models follow the prescribed label mapping of the original dataset, especially for `NLI` datasets. 

For example, the `MNLI` dataset uses the following label mapping:

In [ ]:
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

In [ ]:
from datasets import load_dataset

mnli = load_dataset("glue", "mnli", split="train")
mnli_aligned = mnli.align_labels_with_mapping(label2id, "label")